In [2]:
import Ipynb_importer
from a_csp_darknet53  import *
from b_yolov4_neck_and_body import *
from c_yolov4_head_keras import *
from d_prediction import *

importing Jupyter notebook from d_prediction.ipynb


In [6]:
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
def get_yolo4_model(model_type,  num_anchors, num_classes, input_tensor=None, input_shape=None):
    # prepare input tensor
    if input_shape:
        input_tensor = Input(shape=input_shape, name='image_input')
        
    if input_tensor is None:
        input_tensor = Input(shape=(None, None, 3), name='image_input')
        
    model_function = yolo4_body
    backbone_len = 250
    weights_path = 'weights/cspdarknet53.h5'
    
    if weights_path:
        model_body = model_function(input_tensor, num_anchors//3, num_classes, weights_path=weights_path)
    else:
        model_body = model_function(input_tensor, num_anchors//3, num_classes)
        
    return model_body, backbone_len
    

In [4]:
def get_yolo4_inference_model(model_type, anchors, num_classes, weights_path=None,
                              input_shape=None, confidence=0.1, iou_threshold=0.4, elim_grid_sense=False):
    """create the inference model, for YOLOv4"""
    num_anchors = len(anchors)
    num_feature_layers = num_anchors //3
    
    image_shape = Input(shape=(2,), dtype='int64', name='image_shape')
    model_body, _ = get_yolo4_model(model_type, num_feature_layers, num_anchors, num_classes, input_shape=input_shape)
    
    if weights_path:
        model_body.load_weights(weights_path, by_name=False)
        print('Load weight {}'.format(weights_path))
        
    boxes, scores, classes = Lambda(batch_yolo4_postprocess, name='yolo4_postprocess',
                                    arguments={
                                        'anchors':anchors, 
                                        'num_classes':num_classes, 
                                        'confidence':confidence,
                                        'iou_threshold': iou_threshold,
                                        'elim_grid_sense':elim_grid_sense
                                    })([*model_body.output, image_shape])
    model = Model([model_body.input, image_shape], [boxes, scores, classes])
    
    return model
    
                                        

### 损失层
> https://zhuanlan.zhihu.com/p/42081893

在 Yolov3|4中，损失函数 yolo_loss 封装自定义 Lambad 的损失层中，作为模型的最后一层，参与训练。
损失层 Lambda 的输入是已有模型的输出 yolo_body.output 和真值 y_true，输出是一个值，即损失值。

损失层的核心逻辑位于 yolo_loss 中，yolo_loss 除了接收 Lambda 层的输入 yolo_body.output 和真值 y_true，还接收锚框 anchors、类别数  num_chasses 和 过滤阈值 ignore_thresh 等3个参数。

其中，model_body.output是已有模型的预测值，y_true是真实值，两者的格式相同，如下：

- model_body: [(?, 13, 13, 18), (?, 26, 26, 18), (?, 52, 52, 18)]、
- y_true: [(?, 13, 13, 18), (?, 26, 26, 18), (?, 52, 52, 18)]

In [ ]:
def get_yolo4_train_model(model_type, anchors, num_classes, weights_path=None, freeze_level=1,
                         optimizer=Adam(lr=1e-3, decay=0), label_smoothing=0, elim_grid_sense=False):
    """Create the training model, for YOLOv4"""
    num_anchors = len(anchors)
    num_feature_layers = num_anchors //3
    
    # feature map target value, so its shape should be like:
    # [
    #  (image_height/32, image_width/32, 3, num_classes+5),
    #  (image_height/16, image_width/16, 3, num_classes+5),
    #  (image_height/8, image_width/8, 3, num_classes+5)
    # ]
    y_true = [Input(shape=(None, None, 3, num_classes+5), name='y_true_{}'.format(l)) for l in range(num_feature_layers)]
    
    model_body, backbone_len = get_yolo4_model(model_type, num_feature_layers, num_anchors, num_classes)
    print('Create model')
    
    if weights_path:
        model_body.load_weights(weights_path, by_name=True)
    
    if freeze_level in [1,2]:
        # Freeze the backbone part or freeze all but final feature map & input layers
        num = (backbone_len, len(model_body.layers)-3)[freeze_level-1]  # 通过01选择前面或者后面的数 
        for i in range(num):
            model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))
    elif freeze_level == 0:
        # unfreeze all layers.
        for i in range(len(model_body.layers)):
            model_body.layers[i].trainable = True
        print('Unfree all of the layers. ')
        
    #TODO: yolo4_loss
    model_loss, location_loss, confidence_loss, class_loss = Lambda(yolo3_loss, name='yolo_loss',
                                                                   arguments={
                                                                       'anchors': anchors, 
                                                                       'num_classes': num_classes, 
                                                                       'ignore_thresh': 0.5, 
                                                                       'label_smoothing': label_smoothing, 
                                                                       'elim_grid_sense': elim_grid_sense
                                                                   })([*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)
    
    loss_dict = {'location_loss':location_loss, 'confidence_loss':confidence_loss, 'class_loss':class_loss}
    add_metrics(model, loss_dict)

    model.compile(optimizer=optimizer, loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: y_pred})

    return model